## 実践演習14-3

scikit-learnのLabelPropagationでirisデータの半教師あり学習を行います。

ライブラリの読み込み

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.semi_supervised import LabelPropagation

irisデータの読み込み

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

### 半教師ありデータの作成

まず、どのデータの教師信号を消すかを決めます。
[np.random.choice](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html)を使って、データのインデックスを表すndarrayから、指定された個数のインデックスを、重複なしで抜き出します。

In [3]:
unlabeled_points = np.random.choice(np.arange(y.size), int((y.size)*.7), replace=False)
unlabeled_points

array([ 60,  25,   8, 135,  51, 143, 126,  12,  58, 136, 128, 124, 129,
       107,  15, 140,  63,  56,  95,  79,  29, 102,  59,  52,  31,  72,
       125,  44, 130,  55, 108,  36,   2,  41,  73,  30,  37,  45,  22,
       113,  61,   1,  91,  88,  32,  71,  84,  16,  35,  24,  13, 122,
        78, 104,   9,  92,  83,  54, 116, 101,  75, 110,   3,  53, 142,
        96, 133,  14, 103, 134,  86,  38,  94, 118, 139,  57,  23,  93,
        17, 114, 138,  42,  67,  43,  66,  76,  40,  97,  74,  28, 131,
        10,  34, 144,  69, 146,   0,  47,  18,  85, 148,  82,  90, 123,
        81])

教師ベクトルyを[np.copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.copy.html)を使って変数labelsにコピーし（代入文でコピーすると、オブジェクトが共有されます）、先ほど抜き出したインデックスに対応する値を-1に書き換えます。

In [4]:
labels = np.copy(y)
labels[unlabeled_points] = -1
labels

array([-1, -1, -1, -1,  0,  0,  0,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1,
       -1, -1,  0,  0,  0, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1,  0,
       -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1, -1,  0, -1,  0,  0,  1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1,
        1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  2, -1,
       -1, -1, -1,  2,  2, -1, -1,  2, -1,  2,  2, -1, -1,  2, -1,  2, -1,
        2,  2,  2, -1, -1, -1, -1, -1,  2, -1, -1, -1, -1,  2, -1, -1, -1,
       -1,  2, -1, -1, -1,  2, -1, -1, -1,  2, -1,  2, -1,  2])

これを教師ベクトルとして、[LabelPropagation](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelPropagation.html)で半教師あり学習を行います。

In [5]:
lp = LabelPropagation()
lp.fit(X, labels)

LabelPropagation(alpha=None, gamma=20, kernel='rbf', max_iter=1000, n_jobs=1,
         n_neighbors=7, tol=0.001)

正解なしデータの割り当て結果を表示します。

In [6]:
lp.score(X[unlabeled_points], y[unlabeled_points])

0.9809523809523809

正解付きデータを5%, 10%, 20%, 30%と変えて、それぞれ100回ずつ学習を試みて性能を評価します。

In [7]:
labeled_percent = [0.05, 0.1, 0.2, 0.3]
num = y.size
for labeled in labeled_percent :
    score = 0
    for i in range(100):
        unlabeled_points = np.random.choice(np.arange(num),int(num-num*labeled), replace=False)
        labels = np.copy(y)
        labels[unlabeled_points] = -1
        lp.fit(X, labels)
        score += lp.score(X[unlabeled_points], y[unlabeled_points])
    print("{0}{1:4.1f}{2}{3:6.3f}".format("labeled:", labeled*100, "%, score=", score/100))        

C:\Users\araki\Anaconda3\lib\site-packages\sklearn\semi_supervised\label_propagation.py:288: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


labeled: 5.0%, score= 0.848
labeled:10.0%, score= 0.922
labeled:20.0%, score= 0.951
labeled:30.0%, score= 0.959
